Семинар 2

In [ ]:
!pip install pyspark >> None

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, avg, when, max, month, year, countDistinct
from pyspark.sql import functions as F
import pandas as pd

In [ ]:
spark=SparkSession.builder.appName('Practise').getOrCreate()

df_pyspark = spark.read.load('sales_data.csv', format='csv', sep=',', header='true', Infer_schema=True)

df_pyspark = df_pyspark.withColumn('order_id', df_pyspark['order_id'].cast('int'))
df_pyspark = df_pyspark.withColumn('product_id', df_pyspark['product_id'].cast('int'))
df_pyspark = df_pyspark.withColumn('customer_id', df_pyspark['customer_id'].cast('int'))
df_pyspark = df_pyspark.withColumn('order_date', df_pyspark['order_date'].cast('date'))
df_pyspark = df_pyspark.withColumn('quantity', df_pyspark['quantity'].cast('int'))
df_pyspark = df_pyspark.withColumn('price_per_unit', df_pyspark['price_per_unit'].cast('int'))
df_pyspark = df_pyspark.withColumn('total_price', df_pyspark['total_price'].cast('int'))

df_pyspark = df_pyspark.withColumn('order_month', month(df_pyspark['order_date']))

sales_analysis = df_pyspark.groupBy('order_month').agg(
                                    sum('total_price').alias('total_sales'),
                                    avg('total_price').alias('average_sales'),
                                    max('total_price').alias('max_sales')
                                    )

sales_analysis.orderBy('order_month').show()


Задание 2. Вычислите количество товаров, купленных различными методами оплаты

In [ ]:
sales_analysis_2 = df_pyspark.groupBy('payment_method').agg(sum('quantity').alias('num_of_sales'))
sales_analysis_2.show()

Задание 3. Найдите регион с самой большой суммарной стоимостью продаж

In [ ]:
sales_analysis_3 = df_pyspark.groupBy('region').agg(sum('total_price').alias('total_price_per_region'))
total_price_per_region = sales_analysis_3.select('region').orderBy(sales_analysis_3.total_price_per_region.desc()).first()
print(*total_price_per_region)

Задание 4. Вычислите общую сумму продаж и среднюю сумму продажи для каждого региона

In [ ]:
sales_analysis_4 = df_pyspark.groupBy('region').agg(sum('total_price').alias('total_price_per_region'), avg('total_price').alias('total_price_per_region'))
sales_analysis_4.show()

Задание 5. Вычислите общее количество и сумму товаров, проданных за наличные в 2022 году

In [ ]:
df_pyspark = df_pyspark.withColumn('order_year', year(df_pyspark['order_date']))

sales_analysis_5 = df_pyspark.filter((df_pyspark.payment_method == 'Наличные') & (df_pyspark.order_year == 2022)).agg(
                                      sum('quantity').alias('total_quantity_2022'), sum('total_price').alias('total_price_2022')
                                      )
sales_analysis_5.show()

Задание 6. Найдите уникальное количество покупателей за 2022 год

In [ ]:
sales_analysis_6 = df_pyspark.filter(df_pyspark.order_year == 2022)
sales_analysis_6 = sales_analysis_6.select(countDistinct('customer_id').alias('unique_users'))

sales_analysis_6.show()

Задание 7. Вам даны данные с информацией о стоимости продуктов в различных валютах. Ваша задача состоит в том, чобы перевести все цены в доллары, используя текущие курсы валют. Однако, у вас есть ограничение: для некоторых продуктов курс валюты неизвестен и их стоимость должна остаться в исходной валюте. (Для конвертации из EUR в USD нужно умножить на 1.2)

In [ ]:
spark = SparkSession.builder.appName('currency_conversion').getOrCreate()

data = [(1, 100, 'USD'), (2, 200, 'EUR'), (3, 300, 'Unknown'),
        (4, 100, 'EUR'), (5, 200, 'EUR'), (6, 300, 'Unknown'),
        (7, 100, 'Unknown'), (8, 200, 'USD'), (9, 300, 'USD')]

df_7 = spark.createDataFrame(data, ['product_id', 'price', 'currency'])

df_7 = df_7.withColumn('price_usd', when(df_7.currency == 'USD', df_7.price)
                                .when(df_7.currency == 'USD', df_7.price * 1.2)
                                .otherwise(df_7.price))
df_7.show()

Задание 8. Допустим, есть два датасета: один содержит информацию о пользователях(user_id, name, age), а другой содержит информацию о покупках пользователей (user_id, product_id, date). Необходимо найти  средний возраст пользователей, совершивших покупки

In [ ]:
spark = SparkSession.builder.appName('user_purchase_join').getOrCreate()

users_df = spark.createDataFrame(
          [(1, 'Alice', 25), (2, 'Bob', 30), (3, 'Charlie', 28),
            (4, 'John', 56), (5, 'Alex', 41), (6, 'Juliya', 17)], ['user_id', 'name', 'age']
          )

purchases_df = spark.createDataFrame(
          [(1, 101, '2022-01-01'), (2, 102, '2022-01-02'), (3, 103, '2022-01-03'),
           (3, 104, '2022-01-04'), (6, 105, '2022-01-05')], ['user_id', 'product_id', 'date']
          )

result_df = users_df.join(purchases_df, 'users.id').groupBy('user_id').agg(avg('age').alias('average_age'))

result_df.show()

Задание 9. У вас есть два набора данных. Первый набор содержит информацию о продуктах: id продукта, название, категория и цена. Второй набор содержит информацию о заказах: id заказа, id продукта, количество. Ваша задача - использовать PySpark для выполнения следующих шагов:
1. Присоединить набор данных о продуктах к набору данных о заказах с помощью id продукта
2. Рассчитать общую стоимость каждого заказа, учитывая количество продуктов и их цену.
3. Отфильтровать заказы, у которых стоимость больше 1000

In [ ]:
spark = SparkSession.builder.appName('aggregate_join_filter').getOrCreate()

data_products = [(1, 'product1', 'category1', 10.0), (2, 'product2', 'category2', 15.0), (3, 'product3', 'category1', 12.0),
                (4, 'product4', 'category3', 20.0), (5, 'product5', 'category2', 18.0), (6, 'product6', 'category3', 25.0),
                (7, 'product7', 'category1', 9.0), (8, 'product8', 'category2', 16.0), (9, 'product9', 'category3', 22.0),
                (10, 'product10', 'category1', 11.5)]

products_df = spark.createDataFrame(data_products, ['product_id', 'title', 'category', 'price'])

data_orders = [(1, 1, 5), (2, 3, 2), (3, 2, 3), (4, 5, 1), (5, 4, 4),
               (6, 7, 2), (7, 6, 3), (8, 8, 2), (9, 10, 1), (10, 9, 4)]

orders_df = spark.createDataFrame(data_orders, ['order_id', 'product_id', 'quantity'])

joined_df = orders_df.join(products_df, 'product_id')

total_cost_df = joined_df.withColumn('total_cost', F.col('quantity') * F.col('price'))

filtered_orders_df = total_cost_df.filter(total_cost_df.total_cost > 1000)

filtered_orders_df.show()

Задание 10. Найти сумму чисел в колонке

In [ ]:
spark = SparkSession.builder.appName('sum_example').getOrCreate()

data_10 = [(1,), (2,), (3,), (4,), (5,)]
df_10 = spark.createDataFrame(data_10, ['number'])

sum_result = df_10.select(sum('number'))

sum_result.show()

Задание 11. Посчитать количество уникальных значений в колонке

In [ ]:
spark = SparkSession.builder.appName('count_distinct_example').getOrCreate()

data_11 = [('Alice',), ('Bob',), ('Alice',), ('Eve',)]

df_11 = spark.createDataFrame(data_11, ['name'])
count_result = df_11.select(countDistinct('name'))

count_result.show()

Задание 12. Выполнить фильтрацию данных по определенному условию.

In [ ]:
spark = SparkSession.builder.appName('filter_example').getOrCreate()

data_12 = [('Alice', 25), ('Bob',30 ), ('Eve', 20), ('Charlie', 35)]
df_12 = spark.createDataFrame(data_12, ['name', 'age'])

filtered_data = df_12.filter(df_12.age < 30)

filtered_data.show()

Домашнее задание. Условие: дана таблица с колонками (id, name, salary, manager_id). Студентам необходимо написать код на spark, который создаст эту таблицу (данные указаны ниже) и в результате выдаст таблицу, в которой будут имена сотрудников, которые зарабатывают больше своих менеджеров